In [4]:
from collections import defaultdict
from tabulate import tabulate
import pandas as pd

In [5]:
class HazardDetectionUnit:
    def __init__(self):
        self.PCWrite = True       # When False, PC should not be updated
        self.IF_ID_Write = True   # When False, IF/ID register should not be updated
        self.stall_bubble = False # When True, insert bubble (NOP) in pipeline

    def detect_hazard(self, ID_EX, IF_ID):
        """
        Detect load-use hazards and generate appropriate stall signals.
        Returns: True if hazard detected, False otherwise
        """
        # Reset control signals
        self.PCWrite = True
        self.IF_ID_Write = True
        self.stall_bubble = False

        # Check for load-use hazard
        if ID_EX['MemRead'] and (
            (ID_EX['rt'] == IF_ID['rs']) or
            (ID_EX['rt'] == IF_ID['rt'])):
            # Load-use hazard detected, stall the pipeline
            self.PCWrite = False
            self.IF_ID_Write = False
            self.stall_bubble = True
        print("stall", self.stall_bubble)
        return self.stall_bubble

class ForwardingUnit:
    def __init__(self):
        self.ForwardA = '00'  # No forwarding
        self.ForwardB = '00'  # No forwarding

    def detect_forwarding(self, ID_EX, EX_MEM, MEM_WB):
        """
        ForwardA/B = '00': No forwarding (use value from ID/EX)
        ForwardA/B = '10': Forward from EX/MEM
        ForwardA/B = '01': Forward from MEM/WB
        """
        # Reset forwarding controls
        self.ForwardA = '00'
        self.ForwardB = '00'
        # print( MEM_WB, ID_EX)
        # EX Hazard: Check if EX/MEM stage has a valid forwarding opportunity
        if EX_MEM['RegWrite'] and EX_MEM['destination_register'] != 0:
            if EX_MEM['destination_register'] == ID_EX['rs']:
                self.ForwardA = '10'
            if EX_MEM['destination_register'] == ID_EX['rt']:
                self.ForwardB = '10'
        # print("detect", self.ForwardA )
        # MEM Hazard: Check if MEM/WB stage has a valid forwarding opportunity only if EX/MEM is not forwarding
        if MEM_WB['RegWrite'] and MEM_WB['destination_register'] != 0:
            if MEM_WB['destination_register'] == ID_EX['rs'] and self.ForwardA == '00':
                self.ForwardA = '01'
            if MEM_WB['destination_register'] == ID_EX['rt'] and self.ForwardB == '00':
                self.ForwardB = '01'
        print("forwardA", self.ForwardA)
        print("forwardB", self.ForwardB)

class PipelineRegisters:
    def __init__(self):
        # IF/ID Register
        self.IF_ID = {
            'instruction': 0,  # 32 bits
            'pc_plus4': 0     # 32 bits
        }

        # ID/EX Register
        self.ID_EX = {
            'Func_code': 0, # 6 bits
            'read_data1': 0,# 32 bits
            'read_data2': 0,# 32 bits
            'immediate': 0, # 32 bits
            'destination_register': 0,        # 5 bits
            'rs': 0,        # 5 bits
            'rt': 0,        # 5 bits
            'ALUSrc': 0,    # 1 bit
            'MemRead': 0,   # 1 bit
            'MemWrite': 0,  # 1 bit
            'MemtoReg': 0,  # 1 bit
            'RegWrite': 0,   # 1 bit
            'opcode': 0,

            'branch_target': 0,  # Branch target address
            'Branch': 0,        # Branch control signal
            'jump_target': 0,   # Jump target address
            'Jump': 0,         # Jump control signal
            'is_jump': False,
            'link_address': 0,  # Return address for jal

        }

        # EX/MEM Register
        self.EX_MEM = {
            'zero?':0, #is the output of alu zero?
            'alu_result': 0,    # 32 bits
            'read_data2': 0,    # 32 bits
            'destination_register': 0,     # 5 bits
            'MemRead': 0,   # 1 bit
            'MemWrite': 0,  # 1 bit
            'MemtoReg': 0,  # 1 bit
            'RegWrite': 0,   # 1 bit
            'is_branch': 0,
            'Branch':0,
            'jump_target': 0,   # Jump target address
            'Jump': 0,         # Jump control signal
            'is_jump': False,
            'link_address': 0,  # Return address for jal
        }

        # MEM/WB Register
        self.MEM_WB = {
            'read_data': 0,     # 32 bits
            'alu_result': 0,    # 32 bits
            'destination_register': 0,            # 5 bits
            'MemtoReg': 0,  # 1 bit
            'RegWrite': 0   # 1 bit
        }


        self.MEM_WB_dummy = {
            'read_data': 0,     # 32 bits
            'alu_result': 0,    # 32 bits
            'destination_register': 0,            # 5 bits
            'MemtoReg': 0,  # 1 bit
            'RegWrite': 0   # 1 bit
        }



class MIPSSimulator:
    BIT_MASK=0xFFFFFFFF
    def __init__(self):
        self.pc = 0
        self.flush=0
        self.registers = [0] * 32
        self.registers[28] = 0xFFFFFFA0
        self.memory = defaultdict(lambda: -1e9)
        self.pipeline_regs = PipelineRegisters()
        self.forwarding_unit = ForwardingUnit()
        self.hazard_detection_unit = HazardDetectionUnit()
        self.instruction_memory = [] # list of 32 bit strings
        self.cycle = 0
        self.pipeline_state_history = []
        for i in range(2,28):
          self.registers[i]=i-1
        self.registers[1]=27
        self.registers[29] = 0x00000F9F            # $sp=3999




    def load_program(self, instructions):
        """Load 32-bit instructions into instruction memory"""
        self.instruction_memory = instructions

    def execute_cycle(self):
        """Execute one clock cycle of the pipeline"""
        # Execute stages in reverse order
        print("cycle", self.cycle)
      

        self.pipeline_regs.MEM_WB_dummy= self.pipeline_regs.MEM_WB.copy()
        
        if self.cycle>=4:
            self.write_back_stage()
        if self.cycle>=3:
            self.memory_stage()
        if self.cycle>=2:
            self.execute_stage()
        if self.cycle>=1:
            self.decode_stage()
        if self.cycle>=0:
            self.fetch_stage()

        # Store current state
        self.store_pipeline_state()
        self.cycle += 1

    def write_back_stage(self):
        """WB Stage: Write results back to register file"""
        # Check if RegWrite control signal is set
        if self.pipeline_regs.MEM_WB['RegWrite']:
            # Determine the data to write based on MemtoReg signal
            write_data = (self.pipeline_regs.MEM_WB['read_data']
                          if self.pipeline_regs.MEM_WB['MemtoReg']
                          else self.pipeline_regs.MEM_WB['alu_result']) & self.BIT_MASK

            # Determine which register to write to
            write_reg = self.pipeline_regs.MEM_WB['destination_register']
            if write_reg == 0 and write_data != 0:
                raise ValueError("Error: cannot modify zero register")
            # Perform the write back to the register file
            self.registers[write_reg] = write_data
        if self.pipeline_regs.MEM_WB['link_address']!=-1:
            self.registers[31]= self.pipeline_regs.MEM_WB['link_address']

    def memory_stage(self):
        """MEM Stage: Access data memory"""
        # Move EX/MEM to MEM/WB
        self.pipeline_regs.MEM_WB['alu_result'] = self.pipeline_regs.EX_MEM['alu_result']
        self.pipeline_regs.MEM_WB['destination_register'] = self.pipeline_regs.EX_MEM['destination_register']
        self.pipeline_regs.MEM_WB['MemtoReg']  = self.pipeline_regs.EX_MEM['MemtoReg']
        self.pipeline_regs.MEM_WB['RegWrite']  = self.pipeline_regs.EX_MEM['RegWrite']
        self.pipeline_regs.MEM_WB['link_address']=self.pipeline_regs.EX_MEM['link_address']

        # Memory access
        if self.pipeline_regs.EX_MEM['MemRead']:
            self.pipeline_regs.MEM_WB['read_data'] = self.memory.get(
                self.pipeline_regs.EX_MEM['alu_result'], 0)
        if self.pipeline_regs.EX_MEM['MemWrite']:

            self.memory[self.pipeline_regs.EX_MEM['alu_result']] = (
                self.pipeline_regs.EX_MEM['read_data2'])

    def execute_stage(self):
        """EX Stage: Execute the ALU operation or calculate the branch target address"""
        # Retrieve values from ID/EX pipeline register
        opcode = self.pipeline_regs.ID_EX['opcode']
        func_code = self.pipeline_regs.ID_EX['Func_code']
        read_data1 = self.pipeline_regs.ID_EX['read_data1']
        read_data2 = self.pipeline_regs.ID_EX['read_data2']
        immediate = self.pipeline_regs.ID_EX['immediate']
        rt = self.pipeline_regs.ID_EX['rt']
        destination_register = self.pipeline_regs.ID_EX['destination_register']
        ALUSrc = self.pipeline_regs.ID_EX['ALUSrc']



        # Get forwarding control signals
        self.forwarding_unit.detect_forwarding(
            self.pipeline_regs.ID_EX,
            self.pipeline_regs.EX_MEM,
            self.pipeline_regs.MEM_WB_dummy)

        # Forward input A (first ALU operand)
        if self.forwarding_unit.ForwardA == '10':
            input_A = self.pipeline_regs.EX_MEM['alu_result']
        elif self.forwarding_unit.ForwardA == '01':
            input_A = (self.pipeline_regs.MEM_WB_dummy['read_data']
                      if self.pipeline_regs.MEM_WB_dummy['MemtoReg']
                      else self.pipeline_regs.MEM_WB_dummy['alu_result'])
        else:
            input_A = read_data1

        # Forward input B (before ALUSrc multiplexor)
        if self.forwarding_unit.ForwardB == '10':
            forwarded_B = self.pipeline_regs.EX_MEM['alu_result']
        elif self.forwarding_unit.ForwardB == '01':
            forwarded_B = (self.pipeline_regs.MEM_WB_dummy['read_data']
                          if self.pipeline_regs.MEM_WB_dummy['MemtoReg']
                          else self.pipeline_regs.MEM_WB_dummy['alu_result'])
        else:
            forwarded_B = read_data2

        # ALUSrc multiplexor (after forwarding)
        input_B = immediate if ALUSrc else forwarded_B

        # Initialize ALU result and zero flag
        alu_result = 0
        zero = 0

        # Perform ALU operation based on opcode and function code
        if opcode == 0x00:  # R-type instructions
            if func_code == 0x20:  # add
                alu_result = (input_A + input_B) & self.BIT_MASK
            elif func_code == 0x22:  # sub
                alu_result =( input_A - input_B) & self.BIT_MASK
            elif func_code == 0x24:  # and
                alu_result = (input_A & input_B) & self.BIT_MASK
            elif func_code == 0x25:  # or
                alu_result = (input_A | input_B) & self.BIT_MASK
            elif func_code == 0x2A:  # slt
                alu_result = 1 if input_A < input_B else 0
            elif func_code == 0x00:  # sll
                shamt = input_B & 0x1F# Extract shamt (shift amount) from immediate
                alu_result = (forwarded_B << int(str(shamt), 2)) & self.BIT_MASK # Use forwarded_B for shift operations
            elif func_code == 0x02:  # srl
                shamt = input_B & 0x1F
                alu_result = (forwarded_B >> int(str(shamt), 2) ) & self.BIT_MASK # Use forwarded_B for shift operations
        elif opcode == 0x08:  # addi
            alu_result = (input_A + immediate) & self.BIT_MASK
        elif opcode == 0x0C:  # andi
            alu_result = (input_A & immediate) & self.BIT_MASK
        elif opcode == 0x0D:  # ori
            alu_result = (input_A | immediate) & self.BIT_MASK
        elif opcode == 0x04:  # beq
            alu_result = input_A - forwarded_B
            zero = 1 if alu_result == 0 else 0
        elif opcode == 0x05:  # bne
            alu_result = input_A - forwarded_B
            zero = 1 if alu_result != 0 else 0
        elif opcode == 0x23:  # lw
            alu_result = input_A + immediate  # Address calculation for load
        elif opcode == 0x2B:  # sw
            alu_result = input_A + immediate  # Address calculation for store

        #branch resolution
        if self.pipeline_regs.ID_EX['Branch']:
            if ((opcode == 0x04 and zero == 1) or    # beq and equal
                (opcode == 0x05 and zero == 1)):     # bne and not equal
                # Flush pipeline and update PC

                self.pc = self.pipeline_regs.ID_EX['branch_target']
                self.flush_pipeline()
                self.flush=1
                self.pipeline_regs.EX_MEM['is_branch']=1


        # Update EX/MEM pipeline register
        self.pipeline_regs.EX_MEM['zero?'] = zero
        self.pipeline_regs.EX_MEM['alu_result'] = alu_result
        self.pipeline_regs.EX_MEM['read_data2'] = forwarded_B  # Store forwarded value for sw
        self.pipeline_regs.EX_MEM['destination_register'] = destination_register
        self.pipeline_regs.EX_MEM['MemRead'] = self.pipeline_regs.ID_EX['MemRead']
        self.pipeline_regs.EX_MEM['MemWrite'] = self.pipeline_regs.ID_EX['MemWrite']
        self.pipeline_regs.EX_MEM['MemtoReg'] = self.pipeline_regs.ID_EX['MemtoReg']
        self.pipeline_regs.EX_MEM['RegWrite'] = self.pipeline_regs.ID_EX['RegWrite']
        self.pipeline_regs.EX_MEM['branch_target'] = self.pipeline_regs.ID_EX['branch_target']
        self.pipeline_regs.EX_MEM['Branch'] = self.pipeline_regs.ID_EX['Branch']
        self.pipeline_regs.EX_MEM['link_address']=self.pipeline_regs.ID_EX['link_address']

    def decode_stage(self):
        if self.flush==0:
            """ID Stage: Decode instruction and read registers"""
            # Convert the instruction to a 32-bit binary string
            instruction = format(self.pipeline_regs.IF_ID['instruction'], '032b')

            # Instruction decode
            opcode = (int(instruction, 2) >> 26) & 0x3F
            rs = (int(instruction, 2) >> 21) & 0x1F
            rt = (int(instruction, 2) >> 16) & 0x1F
            destination_register = (int(instruction, 2) >> 11) & 0x1F
            immediate = int(instruction[-16:], 2)
            print("immm", immediate)
            func_code= int(instruction[-6:], 2)
            # Sign extend immediate
            if immediate & 0x8000:
                immediate |= 0xFFFF0000

            jump_target = int(instruction[-26:], 2)

            branch_target = self.pipeline_regs.IF_ID['pc_plus4'] +  (immediate)
            print("target", branch_target, self.pc,  (immediate))

            pc_upper = (self.pipeline_regs.IF_ID['pc_plus4']) & 0xF0000000
            jump_addr = pc_upper | (jump_target)
            print("jump", jump_addr)
            if ( opcode == 0x02 or opcode == 0x03) and (jump_addr-1)<len(self.instruction_memory):
                self.pc=jump_addr
            elif (opcode==0 and func_code==8):
                  self.pc=self.registers[31]
            # Save return address for jal (PC + 4)
            link_address = self.pipeline_regs.IF_ID['pc_plus4']


            # Check for hazards
            IF_ID_fields = {
                'rs': rs,
                'rt': rt
            }

            hazard_detected = self.hazard_detection_unit.detect_hazard(
                self.pipeline_regs.ID_EX,
                IF_ID_fields
            )

            if func_code==8 and opcode==0:
            # Get control signals
                control_signals = self.generate_control_signals(opcode, func_code)
            else:
                control_signals = self.generate_control_signals(opcode)


            # If hazard detected, insert bubble by clearing control signals
            if hazard_detected:
                control_signals = {
                    'RegDst': 0,
                    'ALUSrc': 0,
                    'MemtoReg': 0,
                    'RegWrite': 0,
                    'MemRead': 0,
                    'MemWrite': 0,
                    'Branch': 0,
                    'PCSrc': 0,
                    'ALUOp': 0,
                    'Jump': 0
                }

            # Update ID/EX register
            self.pipeline_regs.ID_EX['read_data1'] = self.registers[rs]
            self.pipeline_regs.ID_EX['read_data2'] = self.registers[rt]
            self.pipeline_regs.ID_EX['immediate'] = immediate
            self.pipeline_regs.ID_EX['rs'] = rs
            self.pipeline_regs.ID_EX['rt'] = rt
            self.pipeline_regs.ID_EX['destination_register'] = destination_register if opcode == 0x00 else rt
            self.pipeline_regs.ID_EX['Func_code'] = int(instruction[-6:], 2)
            self.pipeline_regs.ID_EX['opcode'] = opcode

            # Update control signals (either normal or bubble)
            self.pipeline_regs.ID_EX['ALUSrc'] = control_signals['ALUSrc']
            self.pipeline_regs.ID_EX['MemRead'] = control_signals['MemRead']
            self.pipeline_regs.ID_EX['MemWrite'] = control_signals['MemWrite']
            self.pipeline_regs.ID_EX['RegWrite'] = control_signals['RegWrite']
            self.pipeline_regs.ID_EX['MemtoReg'] = control_signals['MemtoReg']

            #branch signals
            self.pipeline_regs.ID_EX['branch_target'] = branch_target
            self.pipeline_regs.ID_EX['Branch'] = control_signals['Branch']


            # Add jump-specific updates
            self.pipeline_regs.ID_EX['jump_target'] = jump_addr
            self.pipeline_regs.ID_EX['Jump'] = control_signals['Jump']
            self.pipeline_regs.ID_EX['is_jump'] = (opcode == 0x02 or opcode == 0x03)  # j or jal
            self.pipeline_regs.ID_EX['link_address'] = link_address if opcode == 0x03 else -1  # save link address for jal


    def fetch_stage(self):
        if self.flush==0:
            """IF Stage: Fetch instruction from memory"""
            if self.hazard_detection_unit.PCWrite:
                if self.pc < len(self.instruction_memory):
                    self.pipeline_regs.IF_ID['instruction'] = self.instruction_memory[self.pc]
                    self.pipeline_regs.IF_ID['pc_plus4'] = self.pc + 1

                    # Only increment PC if no branch is taken
                    if not (self.pipeline_regs.EX_MEM['is_branch']):
                        self.pc += 1
        else: self.flush=0

    def flush_pipeline(self):
      """Flush the pipeline when branch is taken"""
      # Clear IF/ID register
      self.pipeline_regs.IF_ID['instruction'] = 0
      self.pipeline_regs.IF_ID['pc_plus4'] = 0
      self.pipeline_regs.IF_ID['pc'] = 0


      # Clear ID/EX register
      self.pipeline_regs.ID_EX.update({
          'read_data1': 0,
          'read_data2': 0,
          'immediate': 0,
          'rs': 0,
          'rt': 0,
          'destination_register': 0,
          'ALUSrc': 0,
          'MemRead': 0,
          'MemWrite': 0,
          'RegWrite': 0,
          'MemtoReg': 0,
          'Branch': 0,
          'is_branch': False,
          'branch_target': 0,
          'Func_code': 0,
          'opcode': 0,
          'pc':0
      })

    def generate_control_signals(self, opcode, func_code=0):
        """Generate control signals based on opcode and func_code"""
        # Control signals initialization
        control_signals = {
            'RegDst': 0,
            'ALUSrc': 0,
            'MemtoReg': 0,
            'RegWrite': 0,
            'MemRead': 0,
            'MemWrite': 0,
            'Branch': 0,
            'PCSrc': 0,
            'ALUOp': 0,
            'Jump': 0,
            'is_branch':0
        }

        # Define control signals for each instruction
        if opcode ==0x00 and func_code==8: #jr
            control_signals['jump']=1

        elif opcode == 0x00:  # R-type instructions
            control_signals['RegDst'] = 1
            control_signals['RegWrite'] = 1
            control_signals['ALUOp'] = func_code  # ALU operation based on func_code


        elif opcode == 0x23:  # LW
            control_signals['ALUSrc'] = 1
            control_signals['MemtoReg'] = 1
            control_signals['RegWrite'] = 1
            control_signals['MemRead'] = 1
            control_signals['ALUOp'] = 0  # ADD for address calculation

        elif opcode == 0x2B:  # SW
            control_signals['ALUSrc'] = 1
            control_signals['MemWrite'] = 1
            control_signals['ALUOp'] = 0  # ADD for address calculation

        elif opcode == 0x04:  # BEQ
            control_signals['Branch'] = 1
            control_signals['PCSrc'] = 1
            control_signals['ALUOp'] = 1  # SUB

        elif opcode == 0x05:  # BNE
            control_signals['Branch'] = 1
            control_signals['PCSrc'] = 1
            control_signals['ALUOp'] = 1  # SUB

        elif opcode == 0x08:  # ADDI
            control_signals['ALUSrc'] = 1
            control_signals['RegWrite'] = 1
            control_signals['ALUOp'] = 0  # ADD

        elif opcode == 0x0C:  # ANDI
            control_signals['ALUSrc'] = 1
            control_signals['RegWrite'] = 1
            control_signals['ALUOp'] = 3  # AND (binary 11)

        elif opcode == 0x02:  # J (Jump)
            control_signals['Jump'] = 1

        elif opcode== 0x0D:
            control_signals['ALUSrc'] = 1
            control_signals['RegWrite'] = 1
            control_signals['ALUOp'] = 2  # OR
        
        elif opcode==0x03:  # jal
            control_signals['jump']=1
            control_signals['RegWrite']=1
        
        return control_signals

    def store_pipeline_state(self):
        """Store the current state of the pipeline for later visualization"""
        state = {
            'cycle': self.cycle,
            'pc': self.pc,
            'hazard': self.hazard_detection_unit.stall_bubble,
            'forwardA': self.forwarding_unit.ForwardA,
            'forwardB': self.forwarding_unit.ForwardB,
            'registers': self.registers.copy(),
            'memory': self.memory.copy(),
            'pipeline_regs': {
                'IF_ID': dict(self.pipeline_regs.IF_ID),
                'ID_EX': dict(self.pipeline_regs.ID_EX),
                'EX_MEM': dict(self.pipeline_regs.EX_MEM),
                'MEM_WB': dict(self.pipeline_regs.MEM_WB),
                'MEM_WB_dummy': dict(self.pipeline_regs.MEM_WB_dummy)
            }
        }
        self.pipeline_state_history.append(state)

    def get_pipeline_state_history(self):
      """Return the complete pipeline state history in table format"""
      formatted_history = []
      for state in self.pipeline_state_history:
          cycle_info = f"Cycle {state['cycle']}:\nPC: {hex(state['pc'])}\n"

          # Convert register values to hexadecimal
          register_table = tabulate(
              [(i, hex(reg)) for i, reg in enumerate(state['registers'])],
              headers=["Register", "Value (Hex)"],
              tablefmt="grid"
          )

          # Display pipeline registers in hexadecimal
          if_id_table = tabulate(
              [(k, hex(v) if isinstance(v, int) else v) for k, v in state['pipeline_regs']['IF_ID'].items()],
              headers=["IF/ID", "Value (Hex)"],
              tablefmt="grid"
          )
          id_ex_table = tabulate(
              [(k, hex(v) if isinstance(v, int) else v) for k, v in state['pipeline_regs']['ID_EX'].items()],
              headers=["ID/EX", "Value (Hex)"],
              tablefmt="grid"
          )
          ex_mem_table = tabulate(
              [(k, hex(v) if isinstance(v, int) else v) for k, v in state['pipeline_regs']['EX_MEM'].items()],
              headers=["EX/MEM", "Value (Hex)"],
              tablefmt="grid"
          )
          mem_wb_table = tabulate(
              [(k, hex(v) if isinstance(v, int) else v) for k, v in state['pipeline_regs']['MEM_WB'].items()],
              headers=["MEM/WB", "Value (Hex)"],
              tablefmt="grid"
          )
          mem_wb_dummy_table = tabulate(
              [(k, hex(v) if isinstance(v, int) else v) for k, v in state['pipeline_regs']['MEM_WB_dummy'].items()],
              headers=["MEM/WB_dummy", "Value (Hex)"],
              tablefmt="grid"
          )

          # Convert memory contents to hexadecimal
          memory_table = tabulate(
              [(hex(addr), hex(value)) for addr, value in state['memory'].items()],
              headers=["Memory Address (Hex)", "Value (Hex)"],
              tablefmt="grid"
          )

          # Concatenate all tables and information for the cycle
          cycle_output = (
              f"{cycle_info}\n"
              f"Registers:\n{register_table}\n\n"
              f"IF/ID Register:\n{if_id_table}\n\n"
              f"ID/EX Register:\n{id_ex_table}\n\n"
              f"EX/MEM Register:\n{ex_mem_table}\n\n"
              f"MEM/WB Register:\n{mem_wb_table}\n\n"
              f"MEM/WB_dummy Register:\n{mem_wb_dummy_table}\n\n"
              f"Memory:\n{memory_table}\n\n"
          )
          formatted_history.append(cycle_output)

      # Join all cycle outputs for a complete history printout
      return "\n".join(formatted_history)



    def get_pipeline_state_dataframe(self):
      """Return the complete pipeline state history as a DataFrame."""
      data = []

      # Iterate through each cycle in the history
      for state in self.pipeline_state_history:
          # Dictionary to store the cycle data
          cycle_data = {'Cycle': state['cycle'], 'PC': hex(state['pc']), 'ForwardA': state['forwardA'], 'ForwardB': state['forwardB'], 'hazard': state['hazard']}

          # Add register values in hexadecimal format
          for i, reg_val in enumerate(state['registers']):
              cycle_data[f'Register_{i}'] = hex(reg_val)

          # Add IF/ID, ID/EX, EX/MEM, MEM/WB pipeline registers in hexadecimal format
          for stage in ['IF_ID', 'ID_EX', 'EX_MEM', 'MEM_WB']:
              for key, val in state['pipeline_regs'][stage].items():
                  cycle_data[f'{stage}_{key}'] = hex(val) if isinstance(val, int) else val

          # Add memory contents in hexadecimal format
          for addr, value in state['memory'].items():
              cycle_data[f'Mem_{hex(addr)}'] = hex(value)

          # Append the cycle data to the data list
          data.append(cycle_data)

      # Convert data list to a DataFrame
      df = pd.DataFrame(data)

      # Fill any missing values with None (for cycles where some entries might not exist)
      df = df.fillna('None')

      return df



In [6]:
simulator = MIPSSimulator()



#stalling check
# instructions = [
    # 0x20000007,
    # 0x8C020000,  # lw $2, 20($0)      # Load word into $2
    # 0x8C030018,  # lw $3, 24($0)      # Load word into $3
    # 0x00451020, # add $2, $2, $5     # Add $2 and $5, store result in $2
    # 0x014B4820,  # add $9, $10, $11   # Add $10 and $11, store result in $9
    # 0xAC02001C,  # sw $2, 28($0)      # Store word from $2 into memory at 28($0)
    # # 0xAC030020,  # sw $3, 32($0)      # Store word from $3 into memory at 32($0)
    # 0x2004000A,  # addi $4, $0, 10    # Load immediate 10 into $4
    # 0x20050014,  # addi $5, $0, 20    # Load immediate 20 into $5
    # 0x00852822,  # sub $5, $4, $5     # Subtract $5 from $4, store result in $5
    # 0x00A63824,  # and $7, $5, $6     # Bitwise AND $5 and $6, store result in $7
    # 0x00A63825,  # or $7, $5, $6      # Bitwise OR $5 and $6, store result in $7
    # 0x012A602A,  # slt $12, $9, $10   # Set $12 to 1 if $9 < $10, otherwise 0
    # 0x00044080,  # sll $8, $4, 2      # Shift $4 left by 2, store result in $8
    # 0x00054882,  # srl $9, $5, 2      # Shift $5 right by 2, store result in $9
    # 0x10450003,  # beq $2, $5, 3      # Branch if $2 == $5 (offset 3)
    # 0x14460002,  # bne $2, $6, 2      # Branch if $2 != $6 (offset 2)
    # 0x8C060028,  # lw $6, 40($0)      # Load word into $6 from memory at 40($0)
    # 0xAC06002C,  # sw $6, 44($0)      # Store word from $6 into memory at 44($0)
    # 0x20070005,  # addi $7, $0, 5     # Load immediate 5 into $7
    # 0x00E63820,  # add $7, $7, $6     # Add $7 and $6, store result in $7
    # 0xAC070030,  # sw $7, 48($0)      # Store word from $7 into memory at 48($0)
# ]
###################################swap and store into memory and get elements from memory
# instructions=[
#     0x2004000C,  # addi $4, $0, 12     # Load immediate 12 into $4
#     0x20050005,  # addi $5, $0, 5      # Load immediate 5 into $5
#     0x00804820,  # add $9, $4, $0      # Move $4 to $9 (temporary register for swap)
#     0x00A02020,  # add $4, $5, $0      # Move $5 to $4
#     0x01202820,  # add $5, $9, $0      # Move $9 (original $4) to $5
#     0xAC040000,  # sw $4, 0($0)        # Save swapped $4 to memory at address 0($0)
#     0xAC050004,  # sw $5, 4($0)        # Save swapped $5 to memory at address 4($0)
#     0x8C070000,  # lw $7, 0($0)        # Load from memory at address 0($0) into $7
#     0x8C080004   # lw $8, 4($0)        # Load from memory at address 4($0) into $8
# ]
##############################################branch check
# instructions = [
#     0x20020005,  # addi $2, $0, 5    # $2 = 5
#     0x20030004,  # addi $3, $0, 5    # $3 = 5
#     0x10420002,  # beq $2, $2, 2     # branch if $2 == $3 (taken)
#     0x20020011,  # addi $2, $0, 17    # should be skipped
#     0x20020012,  # addi $2, $0, 18    # should be skipped
#     0x20020013   # addi $2, $0, 19    # branch target
# ]
###############################jump check
# instructions = [
#     0x20020005,  # addi $2, $0, 5       # $2 = 5
#     0x2003000A,  # addi $3, $0, 10      # $3 = 10
#     0x20040014,  # addi $4, $0, 20      # $4 = 20
#     0x08000007,  # j 7                  # jump to instruction at index 7
#     0x20020011,  # addi $2, $0, 17      # should be skipped
#     0x20020012,  # addi $2, $0, 18      # should be skipped
#     0x20020013,  # addi $2, $0, 19      # should be skipped
#     0x2005001E,   # addi $5, $0, 30      # branch target; $5 = 30
#     0x2008000A,
#     0x00084040,
#     0x2008000A
# ]
########################################sll
#instructions=[
    # 0x2008000A,
    # 0x00084040,
    # 0x2008000A]
 #############################################palindrome
instructions = [
0x20080005,
0x20100001,
0x20090001,
0xAE090000,
0x20090007,
0xAE090004,
0x20090003,
0xAE090008,
0x20090002,
0xAE09000C,
0x20090001,
0xAE090010,
0x200A0001,
0x214B0010,
0x200C0001,
0x14B682A,
0x100D0007,
0x8D440000,
0x8D650000,
0x14A40003,
0x214A0004,
0x216BFFFC,
0x800000F,
0x200C0000,
0x200E01A4  # addi $t4, $zero, 0        # set result to 0 (not a palindrome)
]

#######################################jal
# instructions=[
#     0x0C000002, ##jal to 4th instruction
#     0x2004000C,  # addi $4, $0, 12     # Load immediate 12 into $4
#     0x20050005,  # addi $5, $0, 5      # Load immediate 5 into $5
#     0x01F00008
# ]


#forwarding check
# instructions = [
#   0x014B4820,#add $t1, $t2, $t3
#   0x012D6020,#add $t4, $t1, $t5
# ]
simulator.load_program(instructions)
for _ in range(50):
    simulator.execute_cycle()
state_tables= simulator.get_pipeline_state_history()
state_history = simulator.get_pipeline_state_dataframe()

cycle 0
cycle 1
immm 5
target 6 1 5
jump 524293
stall False
cycle 2
forwardA 00
forwardB 00
immm 1
target 3 2 1
jump 1048577
stall False
cycle 3
forwardA 00
forwardB 00
immm 1
target 4 3 1
jump 589825
stall False
cycle 4
forwardA 00
forwardB 00
immm 0
target 4 4 0
jump 34144256
stall False
cycle 5
forwardA 01
forwardB 10
immm 7
target 12 5 7
jump 589831
stall False
cycle 6
forwardA 00
forwardB 01
immm 4
target 10 6 4
jump 34144260
stall False
cycle 7
forwardA 00
forwardB 10
immm 3
target 10 7 3
jump 589827
stall False
cycle 8
forwardA 00
forwardB 01
immm 8
target 16 8 8
jump 34144264
stall False
cycle 9
forwardA 00
forwardB 10
immm 2
target 11 9 2
jump 589826
stall False
cycle 10
forwardA 00
forwardB 01
immm 12
target 22 10 12
jump 34144268
stall False
cycle 11
forwardA 00
forwardB 10
immm 1
target 12 11 1
jump 589825
stall False
cycle 12
forwardA 00
forwardB 01
immm 16
target 28 12 16
jump 34144272
stall False
cycle 13
forwardA 00
forwardB 10
immm 1
target 14 13 1
jump 655361
stall Fa

In [20]:
print(state_tables)

Cycle 0:
PC: 0x1

Registers:
+------------+---------------+
|   Register | Value (Hex)   |
+============+===============+
|          0 | 0x0           |
+------------+---------------+
|          1 | 0x1b          |
+------------+---------------+
|          2 | 0x1           |
+------------+---------------+
|          3 | 0x2           |
+------------+---------------+
|          4 | 0x3           |
+------------+---------------+
|          5 | 0x4           |
+------------+---------------+
|          6 | 0x5           |
+------------+---------------+
|          7 | 0x6           |
+------------+---------------+
|          8 | 0x7           |
+------------+---------------+
|          9 | 0x8           |
+------------+---------------+
|         10 | 0x9           |
+------------+---------------+
|         11 | 0xa           |
+------------+---------------+
|         12 | 0xb           |
+------------+---------------+
|         13 | 0xc           |
+------------+---------------+
|         